In [15]:
import os
import openai
import iris

In [20]:
username = 'demo'
password = 'demo'
hostname = os.getenv('IRIS_HOSTNAME', 'localhost')
port = '1972' 
namespace = 'USER'
CONNECTION_STRING = f"{hostname}:{port}/{namespace}"

In [21]:
# Note: Ideally conn and cursor should be used with context manager or with try-execpt-finally 
conn = iris.connect(CONNECTION_STRING, username, password)
cursor = conn.cursor()

In [22]:
tableName = "Forms.DiagnosticFormFields"
tableDefinition = "(title VARCHAR(255), field_vector VECTOR(DOUBLE, 3072), unit VARCHAR(255), type VARCHAR(255), keywords VARCHAR(1023))"
cursor.execute(f"CREATE TABLE {tableName} {tableDefinition}")

0

In [23]:
import json
import pandas as pd

with open('saved-forms.json', 'r') as file:
  data = json.load(file)

print(data["formFields"])
# Convert the formFields to a DataFrame
df = pd.DataFrame(data["formFields"])


# Create a new column 'description' by combining the relevant fields
df['description'] = df.apply(
  lambda row: f"Title of the field: {row['title']}\nUnit: {row['unit']}\nType: {row['type']}\nKeywords: {', '.join(row['keywords'])}\n", 
  axis=1
)

# Display the DataFrame with the new 'description' column
df.head()

[{'title': 'Datum hodnocení', 'unit': '', 'type': 'date', 'keywords': ['Datum hodnocení']}, {'title': 'Váha', 'unit': 'kg', 'type': 'numeric', 'keywords': ['váha', 'hmotnost', 'váha (kg)', 'hmotnost (kg)']}, {'title': 'Výška', 'unit': 'cm', 'type': 'numeric', 'keywords': ['výška', 'výška (cm)']}, {'title': 'Po linii léčby', 'unit': '', 'type': 'numeric', 'keywords': ['v ____ linii', 'linii léčby', 'x. linie']}, {'title': 'Klinické stádium (Rai)', 'unit': '', 'type': 'text', 'keywords': ['Rai', 'klinické stádium', 'klin. stad.']}]


,title,unit,type,keywords,description
0,Datum hodnocení,,date,[Datum hodnocení],Title of the field: Datum hodnocení\nUnit: \nT...
1,Váha,kg,numeric,"[váha, hmotnost, váha (kg), hmotnost (kg)]",Title of the field: Váha\nUnit: kg\nType: nume...
2,Výška,cm,numeric,"[výška, výška (cm)]",Title of the field: Výška\nUnit: cm\nType: num...
3,Po linii léčby,,numeric,"[v ____ linii, linii léčby, x. linie]",Title of the field: Po linii léčby\nUnit: \nTy...
4,Klinické stádium (Rai),,text,"[Rai, klinické stádium, klin. stad.]",Title of the field: Klinické stádium (Rai)\nUn...


In [26]:
import os
from openai import OpenAI
from dotenv import load_dotenv
load_dotenv()

client = OpenAI()

# Function to create embeddings
def create_embedding(text):
  text = text.replace("\n", " ")
  return openai.embeddings.create(input = [text], model="text-embedding-3-large").data[0].embedding

# Create embeddings for each description
df['embedding'] = df['description'].apply(create_embedding)

# Print the DataFrame with embeddings
df.head()

,title,unit,type,keywords,description,embedding
0,Datum hodnocení,,date,[Datum hodnocení],Title of the field: Datum hodnocení\nUnit: \nT...,"[-0.0130361532792449, 0.012385714799165726, -0..."
1,Váha,kg,numeric,"[váha, hmotnost, váha (kg), hmotnost (kg)]",Title of the field: Váha\nUnit: kg\nType: nume...,"[-0.0197535902261734, 0.0053261942230165005, -..."
2,Výška,cm,numeric,"[výška, výška (cm)]",Title of the field: Výška\nUnit: cm\nType: num...,"[-0.022112155333161354, 0.004534936044365168, ..."
3,Po linii léčby,,numeric,"[v ____ linii, linii léčby, x. linie]",Title of the field: Po linii léčby\nUnit: \nTy...,"[0.0031443366315215826, 0.021565889939665794, ..."
4,Klinické stádium (Rai),,text,"[Rai, klinické stádium, klin. stad.]",Title of the field: Klinické stádium (Rai)\nUn...,"[0.014098881743848324, 0.001271457294933498, -..."


In [34]:
# Prepare the SQL statement and parameters for batch update
sql = f"INSERT INTO {tableName} (title, field_vector, unit, type, keywords) VALUES (?, TO_VECTOR(?), ?, ?, ?)"
params = [(row['title'], str(row['embedding']), row['unit'], row['type'], ','.join(row['keywords'])) for index, row in df.iterrows()]

# Execute the batch update
cursor.executemany(sql, params)

(1, 1, 1, 1, 1)

In [40]:
# Given JSON
new_data = {
  "original_tag": "Hmotnost",
  "key": "Hmotnost",
  "value": "90",
  "type": "number",
  "has_unit": True,
  "unit": "kg"
}

# Create a description string
description = f"Title of the field: {new_data['key']}\nUnit: {new_data['unit']}\nType: {new_data['type']}\nKeywords: {new_data['original_tag']}\n"

# Generate the embedding
new_embedding = create_embedding(description)

# Print the description and embedding
print("Description:", description)
print("Embedding:", new_embedding)

Description: Title of the field: Hmotnost
Unit: kg
Type: number
Keywords: Hmotnost

Embedding: [-0.02013378031551838, 0.004904382396489382, -0.017817627638578415, 0.013862031511962414, 0.0428069569170475, -0.029607679694890976, -0.028324028477072716, 0.03641660884022713, -0.01975705660879612, -0.01585029438138008, 0.04037918150424957, -0.0013019635807722807, -0.016003774479031563, -0.011315658688545227, -0.043672025203704834, 0.012299326248466969, 0.005741545930504799, 0.02196158654987812, -0.04774622246623039, -0.030333220958709717, -0.006801953073590994, -0.020050063729286194, -0.013143465854227543, 0.023203380405902863, 6.267824937822297e-05, -0.003331212792545557, -0.02342662401497364, 0.01787343993782997, 0.03560735285282135, 0.028156597167253494, -0.02863098867237568, -0.012717907316982746, -0.023217331618070602, 0.005033445078879595, 0.008636736311018467, -0.002108605345711112, 0.016324687749147415, 0.023189427331089973, -0.01885012909770012, -0.0169944167137146, 0.0127527900040

In [41]:
sql = f"""
    SELECT TOP ? title, unit, type, keywords
    FROM {tableName}
    ORDER BY VECTOR_DOT_PRODUCT(field_vector, TO_VECTOR(?)) DESC
"""

numberOfResults = 1
cursor.execute(sql, [numberOfResults, str(new_embedding)])

results = cursor.fetchall()
for row in results:
    print(row)

('Váha', 'kg', 'numeric', 'váha,hmotnost,váha (kg),hmotnost (kg)')
